In [2]:
import pandas as pd
import numpy as np
from datetime import date
from workalendar.oceania import Australia, AustralianCapitalTerritory, Victoria, Hobart, NewSouthWales, Queensland, SouthAustralia, Tasmania

In [74]:
import chardet
from glob import glob
import csv
import re

In [2]:
cal = Australia()

In [5]:
cal.holidays(2009)

[(datetime.date(2009, 1, 1), 'New year'),
 (datetime.date(2009, 1, 26), 'Australia Day'),
 (datetime.date(2009, 4, 10), 'Good Friday'),
 (datetime.date(2009, 4, 13), 'Easter Monday'),
 (datetime.date(2009, 4, 27), 'Anzac Day'),
 (datetime.date(2009, 12, 25), 'Christmas Day'),
 (datetime.date(2009, 12, 26), 'Boxing Day'),
 (datetime.date(2009, 12, 28), 'Boxing Day Shift')]

In [133]:
csvs = glob("*.csv")
csvs

['australianpublicholidays-201516.csv',
 'australianpublicholidays-201617.csv',
 'australianpublicholidays-201415.csv',
 'allpubhols.csv',
 'australian_public_holidays_2019.csv',
 'australianpublicholidays-201718.csv']

In [193]:
pubdf = pd.read_csv('allpubhols.csv')
pubdf.head()

,Date,Holiday Name,Information,Applicable To
0,20140101,New Year's Day,New Year's Day is the first day of the calenda...,NAT
1,20140127,Australia Day,Always celebrated on 26 January,NAT
2,20140303,Labour Day,"Always on a Monday, creating a long weekend. I...",WA
3,20140310,March public holiday,The Holidays Act 1910 provides for the third M...,SA
4,20140310,Canberra Day,Held on the second Monday of March each year i...,ACT


In [194]:
pubdf.index = pd.to_datetime(pubdf.Date, format='%Y%m%d')
pubdf.drop(columns='Date', inplace=True)
pubdf.columns = ['HolName','Info','State']
pubdf.head()

,HolName,Info,State
Date,,,
2014-01-01,New Year's Day,New Year's Day is the first day of the calenda...,NAT
2014-01-27,Australia Day,Always celebrated on 26 January,NAT
2014-03-03,Labour Day,"Always on a Monday, creating a long weekend. I...",WA
2014-03-10,March public holiday,The Holidays Act 1910 provides for the third M...,SA
2014-03-10,Canberra Day,Held on the second Monday of March each year i...,ACT


In [195]:
pubdf.State = pubdf.State.str.upper()

In [196]:
pubdf.State.unique()

array(['NAT', 'WA', 'SA', 'ACT', 'TAS', 'VIC', 'NSW|VIC', 'NT',
       'ACT|SA|TAS|VIC|NSW|QLD|NT', 'ACT|NSW|QLD|SA',
       'ACT|TAS|VIC|NSW|QLD|NT', 'ACT|NSW|SA|QLD|NT|VIC', 'ACT|NSW|VIC',
       'ACT|NSW|NT|QLD|SA|TAS|VIC', 'ACT|NSW|SA|VIC|QLD|NT', 'QLD',
       'ACT|NSW|NT|SA|TAS|VIC', 'ACT|NSW|SA', 'ACT|NSW|NT|QLD|WA|TAS|VIC',
       'ACT|TAS|VIC|NSW|QLD|NT|WA', 'ACT|NSW|NT|QLD|SA|VIC',
       'ACT|NSW|QLD|VIC', 'NSW', 'NT|SA', 'ACT|NSW|NT|QLD|TAS|VIC|WA'],
      dtype=object)

In [197]:
pubdf.State = pubdf.State.str.split('|')

In [198]:
pubdf.State.head()

Date
2014-01-01    [NAT]
2014-01-27    [NAT]
2014-03-03     [WA]
2014-03-10     [SA]
2014-03-10    [ACT]
Name: State, dtype: object

In [397]:
dfconv = pubdf.State.apply(lambda rowlist: pd.Series({r:1 for r in rowlist}))
dfconv.head()

,NAT,WA,SA,ACT,TAS,VIC,NSW,NT,QLD
Date,,,,,,,,,
2014-01-01,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-27,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-03-03,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-03-10,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2014-03-10,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN


In [255]:
nat = Australia()
act = AustralianCapitalTerritory()
nsw = NewSouthWales()
qld = Queensland()
sa = SouthAustralia()
tas = Tasmania()
vic = Victoria()
nemstates = [nat, act, nsw, qld, sa, tas, vic]

In [404]:
trydf = pd.concat([pubdf[['HolName','Info']], dfconv[['NAT','ACT','NSW','QLD','SA','TAS','VIC']]], axis=1)
trydf.head()

,HolName,Info,NAT,ACT,NSW,QLD,SA,TAS,VIC
Date,,,,,,,,,
2014-01-01,New Year's Day,New Year's Day is the first day of the calenda...,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-27,Australia Day,Always celebrated on 26 January,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2014-03-03,Labour Day,"Always on a Monday, creating a long weekend. I...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-03-10,March public holiday,The Holidays Act 1910 provides for the third M...,NaN,NaN,NaN,NaN,1.0,NaN,NaN
2014-03-10,Canberra Day,Held on the second Monday of March each year i...,NaN,1.0,NaN,NaN,NaN,NaN,NaN


In [398]:
# trydf = pd.merge(pubdf[['HolName','Info']], dfconv[['NAT','ACT','NSW','QLD','SA','TAS','VIC']], left_index=True, right_index=True)
# trydf.head()

In [405]:
for state in nemstates:
    trydf[f'{state.name}'] = pd.Series(trydf.index).apply(lambda x: not state.is_working_day(x)).astype(float).values
trydf.head()

,HolName,Info,NAT,ACT,NSW,QLD,SA,TAS,VIC,Australia,Australian Capital Territory,New South Wales,Queensland,South Australia,Tasmania,Victoria
Date,,,,,,,,,,,,,,,,
2014-01-01,New Year's Day,New Year's Day is the first day of the calenda...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2014-01-27,Australia Day,Always celebrated on 26 January,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2014-03-03,Labour Day,"Always on a Monday, creating a long weekend. I...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2014-03-10,March public holiday,The Holidays Act 1910 provides for the third M...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0.0,1.0,0.0,0.0,1.0,1.0,1.0
2014-03-10,Canberra Day,Held on the second Monday of March each year i...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,1.0,1.0,1.0


In [406]:
nemcols = trydf.columns[-7:]
nemcols

Index(['Australia', 'Australian Capital Territory', 'New South Wales',
       'Queensland', 'South Australia', 'Tasmania', 'Victoria'],
      dtype='object')

In [407]:
trydf[nemcols].notnull().sum().sum()

2331

In [300]:
trydf.fillna(0, inplace=True)

In [409]:
trydf.insert(0,'Day_of_Week', trydf.index.day_name())
trydf.insert(1, 'daynum', trydf.index.dayofweek)
trydf.head(3)

,Day_of_Week,daynum,HolName,Info,NAT,ACT,NSW,QLD,SA,TAS,VIC,Australia,Australian Capital Territory,New South Wales,Queensland,South Australia,Tasmania,Victoria
Date,,,,,,,,,,,,,,,,,,
2014-01-01,Wednesday,2,New Year's Day,New Year's Day is the first day of the calenda...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2014-01-27,Monday,0,Australia Day,Always celebrated on 26 January,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2014-03-03,Monday,0,Labour Day,"Always on a Monday, creating a long weekend. I...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [410]:
compdict = {k: v for k,v in zip(trydf.columns[4:11], trydf.columns[-7:])}
compdict

{'NAT': 'Australia',
 'ACT': 'Australian Capital Territory',
 'NSW': 'New South Wales',
 'QLD': 'Queensland',
 'SA': 'South Australia',
 'TAS': 'Tasmania',
 'VIC': 'Victoria'}

In [442]:
for k,v in compdict.items():
    print(v, ((trydf[k] == 1) & (trydf[v] == 0) & (trydf.daynum <= 5)).sum())

Australia 0
Australian Capital Territory 0
New South Wales 3
Queensland 9
South Australia 10
Tasmania 7
Victoria 7


In [437]:
changing = trydf.reset_index()
changing.head(2)

,Date,Day_of_Week,daynum,HolName,Info,NAT,ACT,NSW,QLD,SA,TAS,VIC,Australia,Australian Capital Territory,New South Wales,Queensland,South Australia,Tasmania,Victoria
0,2014-01-01,Wednesday,2,New Year's Day,New Year's Day is the first day of the calenda...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,2014-01-27,Monday,0,Australia Day,Always celebrated on 26 January,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [443]:
dferrors = pd.DataFrame(columns=changing.columns)   ### ['Day_of_Week','HolName','Info']
for k,v in compdict.items():
    errors = changing[(changing[k] == 1) & (changing[v] == 0) & (changing['daynum'] <= 5)]
#     print(v)
#     print('------'*10)
    if len(errors) > 0:
        dferrors = pd.concat([dferrors, errors], axis=0)
#         for item in errors.drop_duplicates(['HolName']).items():
#             print(item)
#     print('======'*10)

In [444]:
dferrors

,Date,Day_of_Week,daynum,HolName,Info,NAT,ACT,NSW,QLD,SA,TAS,VIC,Australia,Australian Capital Territory,New South Wales,Queensland,South Australia,Tasmania,Victoria
193,2017-08-07,Monday,0,Bank Holiday,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
225,2018-08-06,Monday,0,Bank Holiday,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
258,2019-08-05,Monday,0,Bank Holiday,Applies to banks and certain financial institu...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19,2014-10-06,Monday,0,Labour Day,"Always on a Monday, creating a long weekend. I...",NaN,1.0,1.0,1.0,1.0,NaN,NaN,0.0,1.0,1.0,0.0,1.0,0.0,0.0
47,2015-10-05,Monday,0,Labour Day,"Always on a Monday, creating a long weekend. I...",NaN,1.0,1.0,1.0,1.0,NaN,NaN,0.0,1.0,1.0,0.0,1.0,0.0,0.0
75,2015-10-05,Monday,0,Labour Day,"Always on a Monday, creating a long weekend. I...",NaN,1.0,1.0,1.0,1.0,NaN,NaN,0.0,1.0,1.0,0.0,1.0,0.0,0.0
103,2016-10-03,Monday,0,Queen's Birthday,Celebrated on First Monday in October in Queen...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.0,1.0,1.0,0.0,1.0,0.0,0.0
134,2016-10-03,Monday,0,Queen's Birthday,Celebrated on First Monday in October in Queen...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.0,1.0,1.0,0.0,1.0,0.0,0.0
167,2017-10-02,Monday,0,Queen's Birthday,Celebrated on First Monday in October in Queen...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.0,1.0,1.0,0.0,1.0,0.0,0.0
198,2017-10-02,Monday,0,Queen's Birthday,Celebrated on First Monday in October in Queen...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.0,1.0,1.0,0.0,1.0,0.0,0.0


<font color = 'purple'><font size = 4>
    Don't care about SA Xmas Eve, or Tassie holidays which only affect a subset of the state.<br>
    Vic AFL grand final seems to be set depending on the AFL season, not sure if there is an actual formula. Will have to just add dates manually.<br>
    NSW August Bank Holiday is banks & certain financial institutions only.<br>
    But Qld Labour Day / Queen's Birthday have changed around in recent years due to political decisions. Might be simpler to manually include these too...

In [451]:
changing.drop(index=dferrors[(dferrors.QLD != 1) & ((dferrors.TAS == 1) | (dferrors.SA == 1) | (dferrors.NSW == 1))].index, inplace=True)
changing

,Date,Day_of_Week,daynum,HolName,Info,NAT,ACT,NSW,QLD,SA,TAS,VIC,Australia,Australian Capital Territory,New South Wales,Queensland,South Australia,Tasmania,Victoria
0,2014-01-01,Wednesday,2,New Year's Day,New Year's Day is the first day of the calenda...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,2014-01-27,Monday,0,Australia Day,Always celebrated on 26 January,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,2014-03-03,Monday,0,Labour Day,"Always on a Monday, creating a long weekend. I...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2014-03-10,Monday,0,March public holiday,The Holidays Act 1910 provides for the third M...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0.0,1.0,0.0,0.0,1.0,1.0,1.0
4,2014-03-10,Monday,0,Canberra Day,Held on the second Monday of March each year i...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,1.0,1.0,1.0
5,2014-03-10,Monday,0,Eight Hours Day,"Always on a Monday, creating a long weekend. I...",NaN,NaN,NaN,NaN,NaN,1.0,NaN,0.0,1.0,0.0,0.0,1.0,1.0,1.0
6,2014-03-10,Monday,0,Labour Day,"Always on a Monday, creating a long weekend. I...",NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0
7,2014-04-18,Friday,4,Good Friday,Easter is celebrated with Good Friday and East...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0
8,2014-04-19,Saturday,5,Easter Saturday,Easter Saturday is between Good Friday and Eas...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0
9,2014-04-20,Sunday,6,Easter Sunday,Public Holiday as part of Easter.,NaN,NaN,1.0,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [453]:
changing.index = changing.Date
changing.drop(columns='Date', inplace=True)
changing

,Day_of_Week,daynum,HolName,Info,NAT,ACT,NSW,QLD,SA,TAS,VIC,Australia,Australian Capital Territory,New South Wales,Queensland,South Australia,Tasmania,Victoria
Date,,,,,,,,,,,,,,,,,,
2014-01-01,Wednesday,2,New Year's Day,New Year's Day is the first day of the calenda...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2014-01-27,Monday,0,Australia Day,Always celebrated on 26 January,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2014-03-03,Monday,0,Labour Day,"Always on a Monday, creating a long weekend. I...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2014-03-10,Monday,0,March public holiday,The Holidays Act 1910 provides for the third M...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0.0,1.0,0.0,0.0,1.0,1.0,1.0
2014-03-10,Monday,0,Canberra Day,Held on the second Monday of March each year i...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,1.0,1.0,1.0
2014-03-10,Monday,0,Eight Hours Day,"Always on a Monday, creating a long weekend. I...",NaN,NaN,NaN,NaN,NaN,1.0,NaN,0.0,1.0,0.0,0.0,1.0,1.0,1.0
2014-03-10,Monday,0,Labour Day,"Always on a Monday, creating a long weekend. I...",NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0
2014-04-18,Friday,4,Good Friday,Easter is celebrated with Good Friday and East...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2014-04-19,Saturday,5,Easter Saturday,Easter Saturday is between Good Friday and Eas...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [454]:
# trydf[['Day_of_Week', 'HolName', 'Info', 'QLD', 'Queensland']][(trydf['QLD'] == 1) & (trydf['Queensland'] == 0) & (trydf.daynum <= 5)]

In [481]:
dfhist_pubhols = pd.DataFrame(index=pd.date_range(start='2009-01-01', end='2018-12-31', freq='D'), columns=['DayName', 'daynum', 'weekend'])
dfhist_pubhols.head()

,DayName,daynum,weekend
2009-01-01,NaN,NaN,NaN
2009-01-02,NaN,NaN,NaN
2009-01-03,NaN,NaN,NaN
2009-01-04,NaN,NaN,NaN
2009-01-05,NaN,NaN,NaN


In [482]:
dfhist_pubhols.tail()

,DayName,daynum,weekend
2018-12-27,NaN,NaN,NaN
2018-12-28,NaN,NaN,NaN
2018-12-29,NaN,NaN,NaN
2018-12-30,NaN,NaN,NaN
2018-12-31,NaN,NaN,NaN


In [483]:
dfhist_pubhols.DayName = dfhist_pubhols.index.day_name()
dfhist_pubhols.daynum = dfhist_pubhols.index.dayofweek
dfhist_pubhols.weekend = dfhist_pubhols.daynum.apply(lambda x: 1 if x > 4 else 0)
dfhist_pubhols.head()

,DayName,daynum,weekend
2009-01-01,Thursday,3,0
2009-01-02,Friday,4,0
2009-01-03,Saturday,5,1
2009-01-04,Sunday,6,1
2009-01-05,Monday,0,0


In [484]:
for state in nemstates:                  ## lambda x: not state.is_working_day(x)
    dfhist_pubhols[f'{state.name}'] = pd.Series(dfhist_pubhols.index).apply(state.is_holiday).astype(float).values
dfhist_pubhols.head()

,DayName,daynum,weekend,Australia,Australian Capital Territory,New South Wales,Queensland,South Australia,Tasmania,Victoria
2009-01-01,Thursday,3,0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2009-01-02,Friday,4,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2009-01-03,Saturday,5,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2009-01-04,Sunday,6,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2009-01-05,Monday,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [363]:
compdict

{'NAT': 'Australia',
 'ACT': 'Australian Capital Territory',
 'NSW': 'New South Wales',
 'QLD': 'Queensland',
 'SA': 'South Australia',
 'TAS': 'Tasmania',
 'VIC': 'Victoria'}

In [414]:
dfconv.head()

,NAT,WA,SA,ACT,TAS,VIC,NSW,NT,QLD
Date,,,,,,,,,
2014-01-01,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-27,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-03-03,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-03-10,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2014-03-10,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN


In [468]:
# dfconv['ACT'][dfconv['ACT'].notnull()]

In [416]:
print(len(pubdf.index))
print(len(dfconv.index))

333
333


In [504]:
notmatching = pd.DataFrame(columns=changing.columns)
for col in ['NAT','ACT','NSW','QLD','SA','TAS','VIC']:   # so for each NEM state I care about
    comparison = changing[changing[col].notnull() & (changing.index.year < 2019) & (changing.daynum < 5)] 
    ## All the AusGov listed public holidays (for this state) I need to check
    
    for day in comparison.index:       # grab the date of the public holiday
        if dfhist_pubhols.loc[day, compdict[col]] == 0:    # look up the same date in dfhist_pubhols, check if it is also a holiday
            notmatching = pd.concat([notmatching, comparison[comparison.index == day]], axis=0)
notmatching.reset_index(inplace=True)
notmatching.drop_duplicates(inplace=True)
notmatching.fillna(0, inplace=True)
notmatching

,index,Day_of_Week,daynum,HolName,Info,NAT,ACT,NSW,QLD,SA,TAS,VIC,Australia,Australian Capital Territory,New South Wales,Queensland,South Australia,Tasmania,Victoria
0,2016-10-03,Monday,0,Labour Day,"Always on a Monday, creating a long weekend. I...",0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
4,2017-10-02,Monday,0,Labour Day,"Always on a Monday, creating a long weekend. I...",0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
8,2018-10-01,Monday,0,Labour Day,"Always on a Monday, creating a long weekend. I...",0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
9,2014-10-06,Monday,0,Labour Day,"Always on a Monday, creating a long weekend. I...",0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
10,2015-10-05,Monday,0,Labour Day,"Always on a Monday, creating a long weekend. I...",0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
14,2016-10-03,Monday,0,Queen's Birthday,Celebrated on First Monday in October in Queen...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
18,2017-10-02,Monday,0,Queen's Birthday,Celebrated on First Monday in October in Queen...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
22,2018-10-01,Monday,0,Queen's Birthday,Celebrated on First Monday in October in Queen...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
23,2015-10-02,Friday,4,AFL GrandFinal,Friday before the AFL Grand Final,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24,2015-10-02,Friday,4,AFL Grand Final,Friday before the AFL Grand Final,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [505]:
notmatching.drop(index=[0,4,8,23], inplace=True)
# unexplained duplicate of 2015 Vic AFL grand final day, and from 2016 QLD had Qns birthday in Oct, no Labour day

In [506]:
notmatching.index = notmatching.iloc[:,0]
notmatching.drop(columns='index', inplace=True)
notmatching

,Day_of_Week,daynum,HolName,Info,NAT,ACT,NSW,QLD,SA,TAS,VIC,Australia,Australian Capital Territory,New South Wales,Queensland,South Australia,Tasmania,Victoria
index,,,,,,,,,,,,,,,,,,
2014-10-06,Monday,0,Labour Day,"Always on a Monday, creating a long weekend. I...",0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
2015-10-05,Monday,0,Labour Day,"Always on a Monday, creating a long weekend. I...",0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
2016-10-03,Monday,0,Queen's Birthday,Celebrated on First Monday in October in Queen...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
2017-10-02,Monday,0,Queen's Birthday,Celebrated on First Monday in October in Queen...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
2018-10-01,Monday,0,Queen's Birthday,Celebrated on First Monday in October in Queen...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
2015-10-02,Friday,4,AFL Grand Final,Friday before the AFL Grand Final,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-09-30,Friday,4,AFL Grand Final,Friday before the AFL Grand Final,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-09-29,Friday,4,AFL Grand Final,Friday before the AFL Grand Final,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-09-28,Friday,4,Friday before the AFL Grand Final,Friday before the AFL Grand Final,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [507]:
notmatching.index.value_counts()

2017-10-02    1
2015-10-05    1
2016-09-30    1
2017-09-29    1
2018-10-01    1
2018-09-28    1
2015-10-02    1
2014-10-06    1
2016-10-03    1
Name: index, dtype: int64

In [508]:
%%time
for col in ['NSW','VIC']:
    for ind in notmatching.index:
#         print(dfhist_pubhols.loc[ind,compdict[col]])
#         print(notmatching.loc[ind,col])
        dfhist_pubhols.loc[ind,compdict[col]] = notmatching.loc[ind,col]

CPU times: user 10.9 ms, sys: 1.95 ms, total: 12.9 ms
Wall time: 11.6 ms


In [509]:
dfhist_pubhols

,DayName,daynum,weekend,Australia,Australian Capital Territory,New South Wales,Queensland,South Australia,Tasmania,Victoria
2009-01-01,Thursday,3,0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2009-01-02,Friday,4,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2009-01-03,Saturday,5,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2009-01-04,Sunday,6,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2009-01-05,Monday,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2009-01-06,Tuesday,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2009-01-07,Wednesday,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2009-01-08,Thursday,3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2009-01-09,Friday,4,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2009-01-10,Saturday,5,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [510]:
dttoday = pd.to_datetime('today').strftime("%Y%m%d")
dfhist_pubhols.to_csv(f'{dttoday}_publicholiday_DT-series_2009-18.csv')